# INSERT screenshots and/or terminal output to demonstrate the process of activating the hadoop cluster
# then adding the files to HDFS

In [4]:
import pyspark

import findspark

from pyspark.sql import SparkSession

import os

# os.environ['SPARK_HOME']

# os.environ['SPARK_HOME'] = '/opt/spark/'

print(os.environ['PATH'])

findspark.init('/opt/spark/')

spark = SparkSession.builder.appName('SparkExample').config('spark.driver.python', '/snap/jupyter/6/bin/python3').getOrCreate()

import sys

print(sys.executable)

print(sys.prefix)

/home/hduser/env/myenv/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/snap/bin:/usr/local/hadoop/bin:/bin:/usr/local/hadoop/sbin:/usr/local/hadoop/bin:/bin:/usr/local/hadoop/sbin:/opt/spark/bin:/usr/local/hadoop/bin:/bin:/usr/local/hadoop/sbin:/opt/spark/bin
/home/hduser/env/myenv/bin/python
/home/hduser/env/myenv


In [17]:
dfs = spark.read.csv("hdfs://localhost:9000/transactions/*.csv", header=True, inferSchema=True)

In [13]:
dfs.show()

+---------+---------+------------+---------+--------+-----+----------+------------+------------+------------+------------+--------------+-----+-----+---------------+-----------+--------------------+-----------+--------------------+----------+------------+------------+
|      _c0|      _c1|         _c2|      _c3|     _c4|  _c5|       _c6|         _c7|         _c8|         _c9|        _c10|          _c11| _c12| _c13|           _c14|       _c15|                _c16|       _c17|                _c18|      _c19|        _c20|        _c21|
+---------+---------+------------+---------+--------+-----+----------+------------+------------+------------+------------+--------------+-----+-----+---------------+-----------+--------------------+-----------+--------------------+----------+------------+------------+
|SHOP_WEEK|SHOP_DATE|SHOP_WEEKDAY|SHOP_HOUR|QUANTITY|SPEND| PROD_CODE|PROD_CODE_10|PROD_CODE_20|PROD_CODE_30|PROD_CODE_40|     CUST_CODE|seg_1|seg_2|      BASKET_ID|BASKET_SIZE|BASKET_PRICE_SEN

In [18]:
dfs.printSchema()

root
 |-- SHOP_WEEK: string (nullable = true)
 |-- SHOP_DATE: string (nullable = true)
 |-- SHOP_WEEKDAY: string (nullable = true)
 |-- SHOP_HOUR: string (nullable = true)
 |-- QUANTITY: string (nullable = true)
 |-- SPEND: string (nullable = true)
 |-- PROD_CODE: string (nullable = true)
 |-- PROD_CODE_10: string (nullable = true)
 |-- PROD_CODE_20: string (nullable = true)
 |-- PROD_CODE_30: string (nullable = true)
 |-- PROD_CODE_40: string (nullable = true)
 |-- CUST_CODE: string (nullable = true)
 |-- seg_1: string (nullable = true)
 |-- seg_2: string (nullable = true)
 |-- BASKET_ID: string (nullable = true)
 |-- BASKET_SIZE: string (nullable = true)
 |-- BASKET_PRICE_SENSITIVITY: string (nullable = true)
 |-- BASKET_TYPE: string (nullable = true)
 |-- BASKET_DOMINANT_MISSION: string (nullable = true)
 |-- STORE_CODE: string (nullable = true)
 |-- STORE_FORMAT: string (nullable = true)
 |-- STORE_REGION: string (nullable = true)



### Perform groupby operations to aggregate data by date for:
##### total spend by store code
##### total spend by store format
##### total spend by prod code / prod_code_10 / prod_code_20 etc - depending on the highest level of granularity during EDA


### Explore EDA capabilities of pyspark - is it better to use pandas on the data directly?
### How useful is spark for feature engineering vs python?
### Is it better to build NN in spark (does spark support keras/tensorflow) or in python?